# Maze Path Finder

In [ ]:
import tkinter as tk
from tkinter import messagebox, ttk
import random
import numpy as np
import json
import threading
import queue
import math

# Default Constants
DEFAULT_WIDTH = 800
DEFAULT_HEIGHT = 600
DEFAULT_GRID_SIZE = 20
DEFAULT_EPISODES = 5000  # Increased for better training

# Q-learning parameters
EPSILON_START = 1.0  # Start with full exploration
EPSILON_END = 0.1    # Minimum exploration
EPSILON_DECAY = 0.995  # Decay rate per episode
GAMMA = 0.95         # Increased discount factor for long-term rewards
ALPHA = 0.1          # Learning rate

class Maze:
    def __init__(self, rows, cols, wall_prob=0.3):
        self.grid = np.ones((rows, cols), dtype=int)  # Initialize all cells as walls
        self.start = None
        self.goal = None
        self.wall_prob = wall_prob

    def generate(self):
        self.grid = np.ones((ROWS, COLS), dtype=int)  # Reset the grid to all walls

        # Ensure start and goal are set
        if self.start is None:
            self.start = (0, 0)
        if self.goal is None:
            self.goal = (ROWS - 1, COLS - 1)

        # Carve a guaranteed path from start to goal
        path = self.carve_path(self.start, self.goal)
        for (x, y) in path:
            self.grid[x][y] = 0

        # Randomly carve additional paths
        for i in range(ROWS):
            for j in range(COLS):
                if self.grid[i][j] == 1:
                    if random.random() > self.wall_prob:
                        self.grid[i][j] = 0  # Make it a path
                    # Else remain a wall

    def carve_path(self, start, goal):
        path = [start]
        current = start
        while current != goal:
            x, y = current
            neighbors = []

            # Possible directions: up, down, left, right
            directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
            random.shuffle(directions)  # Randomize directions

            for dx, dy in directions:
                nx, ny = x + dx, y + dy
                if 0 <= nx < ROWS and 0 <= ny < COLS and (nx, ny) not in path:
                    neighbors.append((nx, ny))

            if not neighbors:
                break  # No way to proceed

            # Choose the neighbor closest to the goal
            neighbors.sort(key=lambda n: abs(n[0]-goal[0]) + abs(n[1]-goal[1]))
            next_cell = neighbors[0]
            path.append(next_cell)
            current = next_cell

        # Ensure the path reaches the goal
        if path[-1] != goal:
            path.append(goal)
        return path

class Robot:
    def __init__(self, maze):
        self.maze = maze
        self.q_table = np.zeros((ROWS, COLS, 4))  # Q-values for each state and action
        self.paths = []  # Store multiple paths
        self.visit_count = np.zeros((ROWS, COLS))  # Track visits to each state
        self.temperature = 1.0  # Initial temperature for Boltzmann exploration

    def choose_action(self, state, epsilon):
        if random.uniform(0, 1) < epsilon:  # Exploration
            return random.randint(0, 3)  # Random action
        else:  # Exploitation with Boltzmann exploration
            q_values = self.q_table[state]
            preferences = q_values / self.temperature
            max_pref = np.max(preferences)
            exp_prefs = np.exp(preferences - max_pref)  # For numerical stability
            probabilities = exp_prefs / np.sum(exp_prefs)
            return np.random.choice(len(q_values), p=probabilities)

    def update_q_value(self, state, action, reward, next_state):
        best_next_action = np.argmax(self.q_table[next_state])  # Max Q-value for the next state
        td_target = reward + GAMMA * self.q_table[next_state][best_next_action]
        td_delta = td_target - self.q_table[state][action]
        self.q_table[state][action] += ALPHA * td_delta  # Update Q-value

    def train(self, episodes, progress_queue):
        epsilon = EPSILON_START
        for episode in range(1, episodes + 1):
            state = self.maze.start
            steps = 0
            path = []
            self.visit_count.fill(0)  # Reset visit count for each episode
            while state != self.maze.goal and steps < ROWS * COLS:
                action = self.choose_action(state, epsilon)
                next_state = self.take_action(state, action)
                path.append(state)

                self.visit_count[state] += 1

                # Reward structure with penalty for multiple visits
                if next_state == self.maze.goal:
                    reward = 100  # Reaching the goal
                elif next_state == state:  # Hit a wall
                    reward = -100
                else:
                    current_distance = abs(state[0] - self.maze.goal[0]) + abs(state[1] - self.maze.goal[1])
                    next_distance = abs(next_state[0] - self.maze.goal[0]) + abs(next_state[1] - self.maze.goal[1])
                    if next_distance < current_distance:
                        reward = 1  # Reward for moving closer
                    elif next_distance > current_distance:
                        reward = -1  # Penalty for moving away
                    else:
                        reward = -0.1  # Small penalty for no change
                    # Additional penalty for revisiting
                    if self.visit_count[next_state] > 1:
                        reward -= 0.5  # Penalty for revisiting

                self.update_q_value(state, action, reward, next_state)
                state = next_state
                steps += 1

            # Decay epsilon
            if epsilon > EPSILON_END:
                epsilon *= EPSILON_DECAY
            else:
                epsilon = EPSILON_END

            # Decay temperature
            self.temperature = max(0.1, self.temperature * 0.995)

            # Store successful paths
            if state == self.maze.goal:
                self.paths.append(path + [self.maze.goal])

            # Send progress update to the queue
            if episode % max(1, episodes // 100) == 0 or episode == episodes:
                progress_queue.put(episode)

            # Optionally, print progress
            if episode % max(1, episodes // 10) == 0 or episode == 1:
                print(f"Training Episode {episode}/{episodes} completed. Epsilon: {epsilon:.4f}")

        # Indicate that training is complete
        progress_queue.put("DONE")

    def take_action(self, state, action):
        x, y = state
        if action == 0:  # Up
            next_state = (max(0, x - 1), y)
        elif action == 1:  # Down
            next_state = (min(ROWS - 1, x + 1), y)
        elif action == 2:  # Left
            next_state = (x, max(0, y - 1))
        else:  # Right
            next_state = (x, min(COLS - 1, y + 1))

        # Check if next_state is a wall
        if self.maze.grid[next_state] == 1:
            return state  # Hit a wall, stay in place
        return next_state

    def get_all_paths(self):
        """Return all unique paths found during training."""
        unique_paths = []
        for path in self.paths:
            if path not in unique_paths:
                unique_paths.append(path)
        return unique_paths

    def get_path(self):
        """Generate path from start to goal using the learned Q-table."""
        path = []
        state = self.maze.start
        visited = set()
        max_steps = ROWS * COLS  # Prevent infinite loops

        for _ in range(max_steps):
            path.append(state)
            visited.add(state)
            action = np.argmax(self.q_table[state])
            next_state = self.take_action(state, action)

            if next_state == state:
                # Hit a wall or no progress, cannot proceed
                print(f"Robot hit a wall at {state}.")
                break

            if next_state == self.maze.goal:
                path.append(next_state)
                print("Robot reached the goal!")
                break

            if next_state in visited:
                # Detected a loop
                print(f"Robot is looping at {next_state}.")
                break

            state = next_state

        return path

    def save_q_table(self, filename):
        with open(filename, 'w') as f:
            json.dump(self.q_table.tolist(), f)

    def load_q_table(self, filename):
        with open(filename, 'r') as f:
            self.q_table = np.array(json.load(f))

class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Maze Solver")
        self.resizable(False, False)

        # Frame for configuration inputs
        config_frame = tk.Frame(self)
        config_frame.pack(pady=10)

        # Input fields for user configuration
        self.label_width = tk.Label(config_frame, text="Width (px):")
        self.label_width.grid(row=0, column=0, padx=5, pady=5, sticky='e')
        self.entry_width = tk.Entry(config_frame, width=10)
        self.entry_width.insert(0, str(DEFAULT_WIDTH))
        self.entry_width.grid(row=0, column=1, padx=5, pady=5)

        self.label_height = tk.Label(config_frame, text="Height (px):")
        self.label_height.grid(row=1, column=0, padx=5, pady=5, sticky='e')
        self.entry_height = tk.Entry(config_frame, width=10)
        self.entry_height.insert(0, str(DEFAULT_HEIGHT))
        self.entry_height.grid(row=1, column=1, padx=5, pady=5)

        self.label_episodes = tk.Label(config_frame, text="Training Episodes:")
        self.label_episodes.grid(row=2, column=0, padx=5, pady=5, sticky='e')
        self.entry_episodes = tk.Entry(config_frame, width=10)
        self.entry_episodes.insert(0, str(DEFAULT_EPISODES))
        self.entry_episodes.grid(row=2, column=1, padx=5, pady=5)

        self.start_button = tk.Button(config_frame, text="Start Maze", command=self.start_maze)
        self.start_button.grid(row=3, column=0, columnspan=2, pady=10)

        # Canvas for maze visualization
        self.canvas = tk.Canvas(self, width=DEFAULT_WIDTH, height=DEFAULT_HEIGHT, bg='white')
        self.canvas.pack()

        # Frame for action buttons
        action_frame = tk.Frame(self)
        action_frame.pack(pady=10)

        self.refresh_button = tk.Button(action_frame, text="Refresh Maze", command=self.refresh_maze)
        self.refresh_button.grid(row=0, column=0, padx=5)

        self.train_button = tk.Button(action_frame, text="Train Robot", command=self.train_robot)
        self.train_button.grid(row=0, column=1, padx=5)

        self.test_button = tk.Button(action_frame, text="Test Robot", command=self.test_robot)
        self.test_button.grid(row=0, column=2, padx=5)

        self.show_paths_button = tk.Button(action_frame, text="Show All Paths", command=self.show_all_paths)
        self.show_paths_button.grid(row=0, column=3, padx=5)

        self.modify_button = tk.Button(action_frame, text="Modify Maze", command=self.modify_maze)
        self.modify_button.grid(row=0, column=4, padx=5)

        # Frame for training progress
        progress_frame = tk.Frame(self)
        progress_frame.pack(pady=10, fill='x')

        self.progress_label = tk.Label(progress_frame, text="Training Progress:")
        self.progress_label.pack(anchor='w', padx=5)

        self.progress_bar = ttk.Progressbar(progress_frame, orient='horizontal', length=600, mode='determinate')
        self.progress_bar.pack(padx=5, pady=5)

        self.episode_label = tk.Label(progress_frame, text="Episode: 0/0")
        self.episode_label.pack(anchor='w', padx=5)

        # Status label
        self.status_label = tk.Label(self, text="Initialize the maze and set start/goal points.", fg="blue")
        self.status_label.pack(pady=5)

        self.maze = None
        self.robot = None
        self.start_set = False  # Track if start point is set

        # Queue for thread communication
        self.progress_queue = queue.Queue()

        # Bind mouse click event
        self.canvas.bind("<Button-1>", self.set_start_or_goal)

    def start_maze(self):
        # Get user inputs
        try:
            width = int(self.entry_width.get())
            height = int(self.entry_height.get())
            episodes = int(self.entry_episodes.get())
        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter valid integers for width, height, and episodes.")
            return

        global ROWS, COLS
        ROWS = height // DEFAULT_GRID_SIZE
        COLS = width // DEFAULT_GRID_SIZE

        self.maze = Maze(ROWS, COLS)
        self.maze.generate()
        self.robot = Robot(self.maze)
        self.draw_maze()

        # Initialize progress bar
        self.progress_bar['maximum'] = episodes
        self.progress_bar['value'] = 0
        self.episode_label.config(text=f"Episode: 0/{episodes}")

        self.status_label.config(text="Maze generated. Click to set START (green) and GOAL (red) points.", fg="blue")

    def draw_maze(self):
        self.canvas.delete("all")  # Clear the canvas
        for i in range(ROWS):
            for j in range(COLS):
                color = 'white' if self.maze.grid[i][j] == 0 else 'black'
                self.canvas.create_rectangle(
                    j * DEFAULT_GRID_SIZE,
                    i * DEFAULT_GRID_SIZE,
                    j * DEFAULT_GRID_SIZE + DEFAULT_GRID_SIZE,
                    i * DEFAULT_GRID_SIZE + DEFAULT_GRID_SIZE,
                    fill=color,
                    outline='gray'
                )
        # Draw start and goal points with unique tags
        if self.maze.start:
            sx, sy = self.maze.start
            self.canvas.create_rectangle(
                sy * DEFAULT_GRID_SIZE,
                sx * DEFAULT_GRID_SIZE,
                sy * DEFAULT_GRID_SIZE + DEFAULT_GRID_SIZE,
                sx * DEFAULT_GRID_SIZE + DEFAULT_GRID_SIZE,
                fill='green',
                tags='start'
            )
        if self.maze.goal:
            gx, gy = self.maze.goal
            self.canvas.create_rectangle(
                gy * DEFAULT_GRID_SIZE,
                gx * DEFAULT_GRID_SIZE,
                gy * DEFAULT_GRID_SIZE + DEFAULT_GRID_SIZE,
                gx * DEFAULT_GRID_SIZE + DEFAULT_GRID_SIZE,
                fill='red',
                tags='goal'
            )

    def refresh_maze(self):
        if self.maze:
            self.maze.generate()  # Generate a new maze
            self.draw_maze()      # Redraw the maze
            if self.robot:
                self.robot.q_table = np.zeros((ROWS, COLS, 4))  # Reset Q-table for new maze
                self.robot.paths = []  # Clear previous paths
            self.start_set = False  # Reset start point setting

            # Reset progress bar and episode label
            try:
                episodes = int(self.entry_episodes.get())
            except ValueError:
                episodes = DEFAULT_EPISODES
                self.entry_episodes.insert(0, str(DEFAULT_EPISODES))
            self.progress_bar['maximum'] = episodes
            self.progress_bar['value'] = 0
            self.episode_label.config(text=f"Episode: 0/{episodes}")

            self.status_label.config(text="Maze refreshed. Click to set START (green) and GOAL (red) points.", fg="blue")
            messagebox.showinfo("Maze Refreshed", "A new maze has been generated. Please set new START and GOAL points.")

    def train_robot(self):
        if not self.robot:
            messagebox.showerror("Error", "Maze not initialized. Please start the maze first.")
            return
        if not self.maze.start or not self.maze.goal:
            messagebox.showerror("Error", "Please set both START and GOAL points before training.")
            return

        try:
            episodes = int(self.entry_episodes.get())
        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter a valid number of episodes.")
            return

        self.progress_bar['value'] = 0
        self.episode_label.config(text=f"Episode: 0/{episodes}")
        self.status_label.config(text="Training in progress...", fg="orange")

        # Disable buttons during training
        self.start_button.config(state='disabled')
        self.refresh_button.config(state='disabled')
        self.train_button.config(state='disabled')
        self.test_button.config(state='disabled')
        self.show_paths_button.config(state='disabled')
        self.modify_button.config(state='disabled')

        # Start training in a separate thread
        training_thread = threading.Thread(target=self.robot.train, args=(episodes, self.progress_queue))
        training_thread.start()

        # Start monitoring the queue
        self.after(100, self.process_queue)

    def process_queue(self):
        try:
            while True:
                msg = self.progress_queue.get_nowait()
                if isinstance(msg, int):
                    # Update progress bar and episode label
                    self.progress_bar['value'] = msg
                    episodes = int(self.entry_episodes.get())
                    self.episode_label.config(text=f"Episode: {msg}/{episodes}")
                elif msg == "DONE":
                    # Training complete
                    self.status_label.config(text="Training complete. You can test the robot now.", fg="green")
                    messagebox.showinfo("Training Complete", "Robot training has been completed.")

                    # Re-enable buttons
                    self.start_button.config(state='normal')
                    self.refresh_button.config(state='normal')
                    self.train_button.config(state='normal')
                    self.test_button.config(state='normal')
                    self.show_paths_button.config(state='normal')
                    self.modify_button.config(state='normal')
        except queue.Empty:
            pass
        finally:
            # Continue checking the queue
            if self.robot and not self.progress_queue.empty():
                self.after(100, self.process_queue)
            elif self.robot:
                self.after(100, self.process_queue)

    def test_robot(self):
        if not self.robot:
            messagebox.showerror("Error", "Robot not initialized. Please start the maze first.")
            return
        if not self.maze.start or not self.maze.goal:
            messagebox.showerror("Error", "Please set both START and GOAL points before testing.")
            return

        path = self.robot.get_path()

        if not path:
            messagebox.showinfo("Result", "No path found.")
            return

        self.status_label.config(text="Testing in progress...", fg="orange")
        self.update_idletasks()

        # Erase previous path by deleting all items with the 'path' tag
        self.canvas.delete('path')

        # Animate the robot moving through the path
        for index, state in enumerate(path):
            x, y = state
            # Calculate color intensity based on the step
            intensity = int(255 * (index / max(len(path)-1, 1)))  # Prevent division by zero
            color = f'#FF{intensity:02x}{intensity:02x}'  # Gradient from red to yellow
            self.canvas.create_rectangle(
                y * DEFAULT_GRID_SIZE,
                x * DEFAULT_GRID_SIZE,
                y * DEFAULT_GRID_SIZE + DEFAULT_GRID_SIZE,
                x * DEFAULT_GRID_SIZE + DEFAULT_GRID_SIZE,
                fill=color,
                outline='',
                tags='path'  # Assign 'path' tag for easy deletion
            )
            self.update()
            self.after(50)  # Adjust speed as needed

        # Highlight the goal
        gx, gy = self.maze.goal
        self.canvas.create_rectangle(
            gy * DEFAULT_GRID_SIZE,
            gx * DEFAULT_GRID_SIZE,
            gy * DEFAULT_GRID_SIZE + DEFAULT_GRID_SIZE,
            gx * DEFAULT_GRID_SIZE + DEFAULT_GRID_SIZE,
            fill='red',
            outline='',
            tags='goal'  # Ensure goal remains highlighted
        )
        self.status_label.config(text="Test complete. Robot has navigated the maze.", fg="green")
        messagebox.showinfo("Test Complete", "Robot has navigated the maze.")

    def show_all_paths(self):
        if not self.robot or not self.robot.paths:
            messagebox.showinfo("No Paths", "No paths have been recorded yet. Please train the robot first.")
            return

        self.status_label.config(text="Displaying all found paths.", fg="green")
        self.canvas.delete('path')  # Remove any existing path visualizations

        colors = ['#FF5733', '#33FF57', '#3357FF', '#F333FF', '#FF33F3', '#33FFF3']  # Different colors for paths
        unique_paths = self.robot.get_all_paths()

        for idx, path in enumerate(unique_paths[:len(colors)]):  # Limit to available colors
            color = colors[idx % len(colors)]
            for state in path:
                x, y = state
                self.canvas.create_rectangle(
                    y * DEFAULT_GRID_SIZE,
                    x * DEFAULT_GRID_SIZE,
                    y * DEFAULT_GRID_SIZE + DEFAULT_GRID_SIZE,
                    x * DEFAULT_GRID_SIZE + DEFAULT_GRID_SIZE,
                    fill=color,
                    outline='',
                    tags='path'
                )
        self.status_label.config(text=f"Displayed {len(unique_paths)} paths.", fg="green")
        messagebox.showinfo("Paths Displayed", f"Displayed {len(unique_paths)} unique paths.")

    def modify_maze(self):
        if not self.maze:
            messagebox.showerror("Error", "Maze not initialized. Please start the maze first.")
            return

        self.status_label.config(text="Click to add/remove walls. Right-click to finish modification.", fg="blue")
        self.canvas.bind("<Button-1>", self.toggle_wall)
        self.canvas.bind("<Button-3>", self.finish_modification)

    def toggle_wall(self, event):
        x = event.y // DEFAULT_GRID_SIZE
        y = event.x // DEFAULT_GRID_SIZE

        if x >= ROWS or y >= COLS:
            return

        # Toggle wall state
        self.maze.grid[x][y] = 0 if self.maze.grid[x][y] == 1 else 1
        self.draw_maze()

        # Ensure start and goal are still paths
        if self.maze.start and self.maze.grid[self.maze.start] == 1:
            messagebox.showwarning("Invalid Start", "Start point is now a wall. Resetting to default.")
            self.maze.start = (0, 0)
        if self.maze.goal and self.maze.grid[self.maze.goal] == 1:
            messagebox.showwarning("Invalid Goal", "Goal point is now a wall. Resetting to default.")
            self.maze.goal = (ROWS - 1, COLS - 1)
        self.draw_maze()

    def finish_modification(self, event):
        self.canvas.unbind("<Button-1>")
        self.canvas.unbind("<Button-3>")
        self.status_label.config(text="Maze modification complete. You can train the robot again.", fg="blue")
        # Reset Q-table for re-training
        if self.robot:
            self.robot.q_table = np.zeros((ROWS, COLS, 4))  # Reset Q-table
            self.robot.paths = []  # Clear previous paths
            try:
                episodes = int(self.entry_episodes.get())
            except ValueError:
                episodes = DEFAULT_EPISODES
                self.entry_episodes.insert(0, str(DEFAULT_EPISODES))
            self.progress_bar['value'] = 0
            self.progress_bar['maximum'] = episodes
            self.episode_label.config(text=f"Episode: 0/{episodes}")
            messagebox.showinfo("Modification Complete", "Maze has been modified. Q-table has been reset. You can train the robot again.")

    def set_start_or_goal(self, event):
        if not self.maze:
            messagebox.showerror("Error", "Maze not initialized. Please start the maze first.")
            return

        # Calculate the grid coordinates
        x = event.y // DEFAULT_GRID_SIZE
        y = event.x // DEFAULT_GRID_SIZE

        if x >= ROWS or y >= COLS:
            # Click outside the maze grid
            return

        if self.maze.grid[x][y] == 1:
            messagebox.showerror("Invalid Selection", "Please select a path (white cell) for START or GOAL.")
            return

        if not self.start_set:
            self.maze.start = (x, y)  # Set start point
            self.start_set = True
            self.status_label.config(text="Start point set. Now, click to set GOAL point (red).", fg="blue")
            print(f"Start point set to: {self.maze.start}")
        else:
            self.maze.goal = (x, y)  # Set goal point
            self.start_set = False
            self.status_label.config(text="Goal point set. You can train the robot now.", fg="blue")
            print(f"Goal point set to: {self.maze.goal}")

        self.draw_maze()  # Redraw the maze to reflect changes

    def run(self):
        self.mainloop()

if __name__ == "__main__":
    app = App()
    app.run()


Start point set to: (1, 39)
Goal point set to: (26, 12)
Training Episode 1/5000 completed. Epsilon: 0.9950
Training Episode 500/5000 completed. Epsilon: 0.1000
Training Episode 1000/5000 completed. Epsilon: 0.1000
Training Episode 1500/5000 completed. Epsilon: 0.1000
Training Episode 2000/5000 completed. Epsilon: 0.1000
Training Episode 2500/5000 completed. Epsilon: 0.1000
Training Episode 3000/5000 completed. Epsilon: 0.1000
Training Episode 3500/5000 completed. Epsilon: 0.1000
Training Episode 4000/5000 completed. Epsilon: 0.1000
Training Episode 4500/5000 completed. Epsilon: 0.1000
Training Episode 5000/5000 completed. Epsilon: 0.1000
Robot reached the goal!
